In [41]:
import sys
import os
import pandas as pd
import glob
sys.path.append(os.path.abspath('..'))

In [42]:
from ingestion.images_parser import extract_text
from extraction.parameter_extraction import extract_parameters
from extraction.patient_info_extractor import extract_patient_info
from models.parameter_interpreter import ParameterInterpreter
from models.pattern_recognition import PatternRecognizer
from models.context_analyzer import ContextAnalyzer

In [43]:
DATA_DIR = "../data/raw_data"
REF_PARAMS_PATH = "../validation/reference_parameter.json"

if not os.path.exists(REF_PARAMS_PATH):
    print(f"Warning: Reference JSON {REF_PARAMS_PATH} not found. Please run the setup script.")

interpreter = ParameterInterpreter(REF_PARAMS_PATH)
context_engine = ContextAnalyzer()
pattern_engine = PatternRecognizer()

In [44]:
files = sorted(glob.glob(os.path.join(DATA_DIR, "*.png")))

print(f"Found {len(files)} available cases.")

TARGET_FILE = "Case_16_High_Cholesterol-Risk.png" 

IMAGE_PATH = os.path.join(DATA_DIR, TARGET_FILE)
print(f"\nSelected Target: {TARGET_FILE}")

Found 20 available cases.

Selected Target: Case_16_High_Cholesterol-Risk.png


In [45]:
print("1. Running OCR...")
raw_text = extract_text(IMAGE_PATH)

print("2. Extracting Patient Profile (Auto)...")
PATIENT_PROFILE = extract_patient_info(raw_text)

print(f"   -> Name:   {PATIENT_PROFILE['name']}")
print(f"   -> Age:    {PATIENT_PROFILE['age']}")
print(f"   -> Gender: {PATIENT_PROFILE['gender']}")
if PATIENT_PROFILE['is_pregnant']:
    print(f"   -> Status: PREGNANT (Will adjust reference ranges)")

print("\n3. Extracting Medical Parameters...")
extracted_data = extract_parameters(raw_text, kb_path=REF_PARAMS_PATH)
print(f"   -> Found {len(extracted_data)} valid parameters.")

1. Running OCR...
2. Extracting Patient Profile (Auto)...
   -> Name:   Heart Harry                                 Age/Gender: 65 Y / Male
   -> Age:    65
   -> Gender: Male

3. Extracting Medical Parameters...
   -> Found 18 valid parameters.


In [46]:
print("4. Interpreting Values...")
analyzed_data = interpreter.analyze(extracted_data)

print("5. Applying Clinical Context...")
refined_data = context_engine.refine_analysis(
    analyzed_data, 
    patient_age=PATIENT_PROFILE['age'], 
    patient_gender=PATIENT_PROFILE['gender'], 
    is_pregnant=PATIENT_PROFILE['is_pregnant']
)

print("6. Detecting Disease Patterns...")
risks = pattern_engine.evaluate_risks(refined_data)

4. Interpreting Values...
5. Applying Clinical Context...
6. Detecting Disease Patterns...


In [47]:
print("\n" + "="*60)
print(f"ANALYSIS REPORT: {PATIENT_PROFILE['name']}")
print("="*60)

df = pd.DataFrame(refined_data)
if not df.empty:
    cols = ['Standard_Name', 'Value', 'Unit', 'Flag', 'Context_Note']
    existing_cols = [c for c in cols if c in df.columns]
    display(df[existing_cols])
else:
    print("No parameters extracted.")

print(f"\n[DIAGNOSTIC ALERTS]")
if risks:
    for r in risks:
        level = r['Risk'].upper()
        print(f"[{level}] {r['Condition']}")
else:
    print("No specific disease patterns detected.")


ANALYSIS REPORT: Heart Harry                                 Age/Gender: 65 Y / Male


,Standard_Name,Value,Unit,Flag,Context_Note
0,Hemoglobin,16.2,gfdb,Within Limits,
1,RBC Count,5.3,millfcumm,Within Limits,
2,Total WBC,7277.0,/cumm,Within Limits,
3,Platelet Count,342409.0,/cumm,Within Limits,
4,MCV,90.3,fL,Within Limits,
5,Neutrophils,54.4,Si,Within Limits,
6,Lymphocytes,34.1,Si,Within Limits,
7,Glucose,70.6,mg/dl,Within Limits,
8,Blood Urea,20.1,mg/dl,Within Limits,
9,Creatinine,0.8,mg/dl,Within Limits,



[DIAGNOSTIC ALERTS]
[HIGH] Dyslipidemia (High Cholesterol Risk)
